In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/facial-keypoints-detection/training.zip
/kaggle/input/facial-keypoints-detection/SampleSubmission.csv
/kaggle/input/facial-keypoints-detection/IdLookupTable.csv
/kaggle/input/facial-keypoints-detection/test.zip


In [67]:
import sklearn

In [68]:
train = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip')
test = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip')
lookid_data = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')

In [69]:
train.head().T

,0,1,2,3,4
left_eye_center_x,66.033564,64.332936,65.057053,65.225739,66.725301
left_eye_center_y,39.002274,34.970077,34.909642,37.261774,39.621261
right_eye_center_x,30.227008,29.949277,30.903789,32.023096,32.24481
right_eye_center_y,36.421678,33.448715,34.909642,37.261774,38.042032
left_eye_inner_corner_x,59.582075,58.85617,59.412,60.003339,58.56589
left_eye_inner_corner_y,39.647423,35.274349,36.320968,39.127179,39.621261
left_eye_outer_corner_x,73.130346,70.722723,70.984421,72.314713,72.515926
left_eye_outer_corner_y,39.969997,36.187166,36.320968,38.380967,39.884466
right_eye_inner_corner_x,36.356571,36.034723,37.678105,37.618643,36.98238
right_eye_inner_corner_y,37.389402,34.361532,36.320968,38.754115,39.094852


In [70]:
test.head()

,ImageId,Image
0,1,182 183 182 182 180 180 176 169 156 137 124 10...
1,2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
2,3,177 176 174 170 169 169 168 166 166 166 161 14...
3,4,176 174 174 175 174 174 176 176 175 171 165 15...
4,5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [71]:
train.isnull().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [72]:
train.fillna(method = 'ffill', inplace = True)
#inplace = modify the original df

/tmp/ipykernel_36/3031855114.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train.fillna(method = 'ffill', inplace = True)


In [73]:
test.isnull().sum()

ImageId    0
Image      0
dtype: int64

In [ ]:
imag = [] #list to store all processed values
for i in range(0, 7049): #loop through all 7049 train samples
    img_str_list = train['Image'][i].split(' ') #takes every image value and splits the pixels into a new list


    processed_img_list = [] #list to store clean values
    for x in img_str_list: #for every number in pixel brightnesses
        if x == '': #if there is an empty value
            processed_img_list.append('0') #add 0 in its place
        else:
            processed_img_list.append(x) #else add the original value

    imag.append(processed_img_list) #add the clean values to imag list

In [ ]:
imag

In [ ]:


image_to_list = np.array(imag,dtype = 'float')
X_train = image_to_list.reshape(-1,96,96,1)

plt.imshow(X_train[10].reshape(96,96),cmap='gray')
plt.show()



In [ ]:
training= train.drop("Image", axis= 1)
y_train = []
for i in range(0,7049):
    y = training.iloc[i,:].values

    y_train.append(y)
y_train = np.array(y_train,dtype = 'float')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,Dropout,Dense,Flatten,BatchNormalization ,MaxPool2D
from keras.layers import LeakyReLU

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',
                  input_shape = (96,96,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same' 
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' 
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same' 
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))



model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' 
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same' 
                 ))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(30))
model.summary()

In [ ]:


model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])



In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=256, validation_split = 0.15)

In [ ]:
timag = []
for i in range(0, 1783):
    img_str_list = test['Image'][i].split(' ')
    
   
    processed_img_list = [] 
    for x in img_str_list: 
        if x == '':
            processed_img_list.append('0')
        else:
            processed_img_list.append(x)
    
    timag.append(processed_img_list) 

In [ ]:
timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1) 

In [ ]:
predict = model.predict(X_test)

In [ ]:


lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(predict)



In [ ]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [ ]:


feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))



In [ ]:


preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])



In [ ]:
rowid = pd.Series(rowid,name = 'RowId')

In [ ]:
loc = pd.Series(preded,name = 'Location')

In [ ]:
submission = pd.concat([rowid,loc],axis = 1)

In [ ]:
submission.to_csv('facial_detection_submission.csv',index = False)